## Script to assemble DECam images for SBF g-z color measurements.
Original version Ehsan Kourkchi, Fall 2023.
This version limits cutout size to 1500<r<5000 pixels, Joe Jensen, 5 June 2024.
Use default minimum image size when PGC ID or d25 is not found in the LEDA database.

In [1]:
import os, sys

pysbf_path = "/Users/Joe/data/sbf/"
sys.path.insert(0, pysbf_path)
from pysbf import *

#DECam_data = "/Users/Joe/data/DECam/"

In [2]:
import astropy.io.fits as fits
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pylab as py

In [3]:
def xcmd(cmd,verbose):

    if verbose: print('\n'+cmd)

    tmp=os.popen(cmd)
    output=''
    for x in tmp: output+=x
    if 'abort' in output:
        failure=True
    else:
        failure=tmp.close()
    if False:
        print('execution of %s failed' % cmd)
        print('error is as follows',output)
        sys.exit()
    else:
        return output
    
from contextlib import contextmanager
import os

@contextmanager
def cwd(path):
    oldpwd = os.getcwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(oldpwd)
        
        
class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [4]:
def add_hdr(hdr, header, key):
    
    value = header[key]
    
    if isinstance(value, str):
        value = "'" + value + "'"
    else:
        value = str(value)
    
    hdr += key + "=" + value + " / " + header.comments[key] + "\n"
    
    return hdr

In [5]:
def getPGCid(objname):
    """extracting the PGC ID of a galaxy given its name

    Args:
        objname  (str): galaxy name

    Returns:
        int: PGC ID
    """

    
    my_url = 'http://ned.ipac.caltech.edu/cgi-bin/nph-objsearch?objname='+objname+'&extend=no&of=xml_names&extend=no&hconst=73&omegam=0.27&omegav=0.73&corr_z=1&out_csys=Equatorial&out_equinox=J2000.0&obj_sort=RA+or+Longitude&of=pre_text&zv_breaker=30000.0&list_limit=5&img_stamp=YES'
    tag = mySoup(my_url).findAll('a', href="/cgi-bin/catdef?prefix=PGC")[0].parent
    pgc = int(str(tag).split("</a>")[1].split('</td>')[0].strip())
   
    return pgc

In [6]:
def addGalaxyInfo(Leda, pgc, response=None, objname=None):
    """Exctracting the information of a galaxy from the LEDA catalog 
    given the corresponsing PGC ID

    :param Leda: Leda catalog
    :type Leda: ``Pandas dataFrame``
    :param pgc: pgc ID
    :type pgc: ``int``
    :param response: response to be modified, defaults to None
    :type response: python ``dictionary``, optional
    :param objname: galaxy name, defaults to None
    :type objname: ``str``, optional
    :return: response dictionary that holds the requested galaxy information
    :rtype: python dictionary
    """

    if response is None:
        response = {}

    myGalaxy = Leda.loc[int(pgc)]
    response["pgc"]     = pgc
    response["ra"]      = "%.4f"%((myGalaxy.al2000)*15)         # deg
    response["dec"]     = "%.4f"%(myGalaxy.de2000)              # deg
    response["fov"]     = "%.2f"%(1.5*0.1*10**myGalaxy.logd25)  # arcmin
    response["pa"]      = str(myGalaxy.pa)

    if objname is None:
        response["objname"] = str(myGalaxy.objname)
    else:
        response["objname"] = objname

    return response

In [7]:
def loadLeda(leda_catalog):
    """Loading the HyperLeda catalog: 
    - http://leda.univ-lyon1.fr/
    - This catalog tabulates the proper information on local galaxies

    :return: the Leda catalog in the Pandas dataFrame format
    :rtype: Pandas ``dataFrame``
    """

    df = pd.read_csv(leda_catalog, delimiter=',')
    df = df.rename(columns=lambda x: x.strip())
                   
    return df.set_index('PGC')

In [8]:
Leda = loadLeda("LEDA.csv")

Leda.head()

/var/folders/kj/1b979sh90n3_hb8hr85cl4n00000gq/T/ipykernel_34462/963424436.py:10: DtypeWarning: Columns (71,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(leda_catalog, delimiter=',')


,al1950,de1950,al2000,de2000,l2,b2,sgl,sgb,t,e_t,...,objname,objtype,type,bar,ring,multiple,compactness,angclass,RA2000,DEC2000
PGC,,,,,,,,,,,,,,,,,,,,,
1,23.95695,0.42327,23.99966,0.70168,96.87946,-59.54180,293.35019,13.43002,2.4,5.0,...,PGC000001,G,S?,NaN,NaN,NaN,NaN,NaN,235958.8,4206.0
2,23.95786,46.99616,0.00047,47.27450,113.95535,-14.69916,341.64403,20.73880,3.1,0.4,...,UGC12889,G,Sb,B,NaN,NaN,NaN,NaN,1.7,471628.2
4,23.95829,22.80927,0.00096,23.08764,107.83219,-38.27289,316.05865,18.45140,5.0,1.2,...,PGC000004,G,Sc,NaN,NaN,NaN,NaN,NaN,3.5,230515.5
5,23.95988,32.46003,0.00254,32.73839,110.62060,-28.90431,326.17705,19.78055,-0.6,1.2,...,IC5370,G,S0-a,NaN,NaN,NaN,NaN,NaN,9.1,324418.2
6,23.95790,15.60327,0.00058,15.88165,105.25252,-45.21822,308.62424,17.10314,-1.0,2.0,...,PGC000006,G,S0-a,NaN,NaN,NaN,C,NaN,2.1,155253.9


In [9]:
def load_info(galaxy_name, default_size = 0):
    
    # import pdb; pdb.set_trace()
    
    ned = get_ned_info([galaxy_name])
    
    try:
        if ned is not None:
            objName = ned.iloc[0]["Object Name"]
            ra, dec = ned.iloc[0]["ra_dec"]

            try:
                pgc = getPGCid(objName.replace(" ", "%20"))
            except:
                pgc = "None"
                
            try:
                logd25 = Leda.loc[pgc]["logd25"]
            except: 
                logd25 = default_size
                            
            return pgc, objName.replace(" ", ""), ra, dec, logd25
    except: 
        print(f"Couldn't load data for {galaxy_name}. Using default image size.")
        pass
        
    return None

In [10]:
def get_decam(DECam_data, bricks, pgc, objName, ra, dec, logd25):

    #import pdb; pdb.set_trace()
    
    temp_repo_name = DECam_data + objName
    d25 = 10**logd25
    size = 0.1 * d25 / 60  # deg
    delta = size * 8
    pix_size = 0.262   # arcsec
    naxis = int(8*size*3600 / pix_size)
    
    if naxis > 5000:  # If the image is really big, just use max size of 5000 pix
        naxis=5000    # pixels for cutout
        delta = 0.365   # deg (5000 pix = 0.365 deg)
    if naxis < 1500 or d25==0:  # If the image is small, just use min size of 1500 pix
        naxis=1500    # pixels for cutout
        delta = 0.109   # deg (1500 pix = 0.109 deg)
    print('naxis = ',naxis)

    ra_max  =  ra+delta
    ra_min  =  ra-delta
    dec_max =  dec+delta
    dec_min =  dec-delta


    dg = bricks[(bricks["RA1"]<ra_max) & 
       (bricks["RA2"]>ra_min) &
       (bricks["DEC1"]<dec_max) & 
       (bricks["DEC2"]>dec_min) 
      ]

    if not os.path.isdir(temp_repo_name):
        os.mkdir(temp_repo_name)

    for filt in ["g", "z"]:

        urls = [] 
        for iid in dg.BRICKNAME.values:

            s = str(iid)[2:-1]

            base_url = f"https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/{s[:3]}/{s}/"
            image_name = f"legacysurvey-{s}-image-{filt}.fits.fz"
            url = base_url + image_name

            urls.append(url)

        with cwd(temp_repo_name):

            for url in urls:
                xcmd(f"wget {url}", verbose=False)

            cmd = 'mkdir tmp'
            xcmd(cmd, False)

            for url in urls:

                fname = url.split("/")[-1].split(".")[0]
                hdulist = fits.open(fname+".fits.fz")
                prihdr = hdulist[1].header 

                fits.writeto('tmp/' + fname+'.fits', hdulist[1].data, header=hdulist[1].header, 
                                        output_verify='exception', overwrite=True, checksum=False)

                fout=open('montage.csh','w')

            montage = '''
              rm *tbl
              rm -rf projected
              mkdir  projected
              mImgtbl tmp rimages.tbl
              mProjExec -p tmp rimages.tbl tmp.hdr projected stats.tbl
              mImgtbl projected pimages.tbl
              mAdd -p projected pimages.tbl tmp.hdr tmp.fits
              mConvert -b -32 tmp.fits tmp.fits
              rm *area*fits
              rm *tbl
              rm -rf projected
              '''

             # setenv PATH $PATH":/Users/Joe/src/Montage/bin"

            fout.write(montage+'\n')
            fout.close()

            crpix = naxis/2
            hdr = ''
            hdr += 'SIMPLE  = T' + '\n'
            hdr += 'BITPIX  = -32' + '\n'
            hdr += 'NAXIS   = 2' + '\n'
            hdr += 'NAXIS1  = '+str(naxis) + '\n'
            hdr += 'NAXIS2  = '+str(naxis) + '\n'
            hdr += 'CTYPE1  = \'RA---TAN\'' + '\n'
            hdr += 'CTYPE2  = \'DEC--TAN\'' + '\n'
            hdr += 'CRVAL1  = ' +str(ra)+ '\n'
            hdr += 'CRVAL2  = ' +str(dec)+ '\n'
            hdr += 'CRPIX1  = '+str(crpix) + '\n'
            hdr += 'CRPIX2  = '+str(crpix) + '\n'
            hdr += 'CDELT1  = -7.27777778E-05' + '\n'
            hdr += 'CDELT2  = 7.27777778E-05' + '\n'
            hdr += 'CROTA2  = 0.000000' + '\n'
            hdr += 'EQUINOX = 2000.0' + '\n'
            hdr += 'BSCALE  = 1' + '\n'
            hdr += 'BZERO   = 0' + '\n'
            hdr += 'EXPTIME = 1.0' + '\n'
            hdr += 'ZP      = 16.40006562 ' + '\n'
            hdr = add_hdr(hdr, prihdr, "MAGZERO")
            hdr = add_hdr(hdr, prihdr, "BUNIT")
            hdr = add_hdr(hdr, prihdr, "COSKY_"+filt.upper())
            hdr = add_hdr(hdr, prihdr, "FILTER")
            hdr = add_hdr(hdr, prihdr, "FILTERX")
            hdr = add_hdr(hdr, prihdr, "INSTRUME")
            hdr = add_hdr(hdr, prihdr, "TELESCOP")
            hdr = add_hdr(hdr, prihdr, "OBSERVAT")
            hdr = add_hdr(hdr, prihdr, "EQUINOX")
            hdr = add_hdr(hdr, prihdr, "LSDR")
            hdr = add_hdr(hdr, prihdr, "SURVEY")
            hdr = add_hdr(hdr, prihdr, "SURVEYID")
            hdr = add_hdr(hdr, prihdr, "LEGPIPEV")
        #     hdr = add_hdr(hdr, prihdr, "")
        #     hdr = add_hdr(hdr, prihdr, "")

            hdr += 'PGC = ' + 'PGC'+str(pgc) + '\n'
            hdr += 'OBJECT = ' + objName + '\n'
            hdr += 'HISTORY = \'email: <ehsan20@hawaii.edu>\'' + '\n'  
            hdr += 'END' + '\n'

            fout=open('tmp.hdr','w')
            fout.write(hdr)
            fout.close()

            xcmd('csh montage.csh', False)
            xcmd('rm montage.csh', False)
            xcmd('rm tmp.hdr', False)
            xcmd('rm -rf tmp', False)
            xcmd('rm -rf *.fits.fz*', False)
            xcmd(f'mv tmp.fits {objName}_{filt}.fits', True)

In [11]:
from astropy.table import Table

data = Table.read('survey-bricks.fits', format='fits')
bricks = data.to_pandas()

In [16]:
# ACS color calibration galaxies:
# DECam_data = "/Users/Joe/data/DECam/"
# galaxies = [
# "ic0798",
# "ic0809",
# "ic3019",
# "ic3025",
# "ic3032",
# "ic3065",
# "ic3101",
# "ic3292",
# "ic3328",
# "ic3381",
# "ic3383",
# "ic3442",
# "ic3461",
# "ic3468",
# "ic3470",
# "ic3487",
# "ic3490",
# "ic3492",
# "ic3501",
# "ic3509",
# "ic3586",
# "ic3602",
# "ic3612",
# "ic3633",
# "ic3635",
# "ic3647",
# "ic3652",
# "ic3653",
# "ic3735",
# "ic3773",
# "ic3779",
# "n4262",
# "n4267",
# "n4309a",
# "n4318",
# "n4340",
# "n4350",
# "n4352",
# "n4371",
# "n4377",
# "n4379",
# "n4387",
# "n4417",
# "n4434",
# "n4435",
# "n4452",
# "n4458",
# "n4464",
# "n4467",
# "n4472",
# "n4474",
# "n4476",
# "n4478",
# "n4479",
# "n4482",
# "n4483",
# "n4486a",
# "n4486b",
# "n4489",
# "n4515",
# "n4528",
# "n4550",
# "n4551",
# "n4564",
# "n4570",
# "n4578",
# "n4612",
# "n4623",
# "n4638",
# "n4649",
# "n4660",
# "u7399a",
# "u7436",
# "u7580",
# "u7584",
# "n4874",
# "n4993",
# "ic1919",
# "ic2006",
# "n1344",
# "n1374",
# "n1375",
# "n1380",
# "n1399",
# "n1404"

# -------SN-31 galaxies (17446)
#DECam_data = "/Users/Joe/data/wfc3-17446/DECam/" 
#galaxies = [
# "cgcg-005-038",
# "cgcg-031-049", 
# "cgcg-285-013", # no g-band images
# "eso352-g057", # didn't work, Should have data
# "eso442-g015",
# "eso479-g007",
# "ic0511", # no g-band images
# "mcg-02-33-017",
# "cgcg-097-050",
# "mcg+08-07-008", # no DECam data
# "u00402",
# "n0083",
# "n1209",
# "n3332",
# "n3643",
# "n3941", # no g images.
# "n4125", # no g images.
# "n4169",
# "n4415",
# "n4636",
# "n4767",
# "n5018",
# "n5222",
# "n5304",
# "n5419",
# "n5631", # no g images
# "n7187",
# "leda-1693718", # No DECam images
# "u02829"   

# -------SNAP-28 galaxies (16262)
#DECam_data = "/Users/Joe/data/wfc3-16262/DECam/"
#galaxies = [
# "WISEAJ192159.44-600234.4", # aka 2MASS-J19215490-6003146
# "WISEAJ125011.54-411315.5", # aka "2MASS-J12501152-4113155",
##"eso137-g006", # should have data
##"eso137-g008", # should have data
# "eso185-g054",
# "eso235-g049",
##"eso399-g010", # 404 error
# "ic4727", # no z data
# "ic4794",
# "ic4798",
##"ic4926", # no z image? There should be. (404 error)
##"ic4931", # no z image? There should be (404)
# "ic4956",
##"ic4991", # didn't complete -> idk
# "ic5105",
##"ic5193", # didn't work
##"n0439", # should work. --> 
# "n1490",
# "n3250", # no images
# "n3268", # no g images
##"n3283", # should have data.
# "n4373",    
##"n5090", # should have data. 
# "n6407", # no data or no z data
# "n6653",    
# "n6739", # bad stack, but we will make do
# "n6768",
##"n6841", # should have data.
# "n6861",
# "n6868",
# "n7014",
# "pgc63905",

# "cgcg328-014", # no DECam images
# "cgcg539-126", # no images
# "cgcg-540-079", # no images
##"eso436-g045",
# "eso461-g007", # no z data
# "eso462-g015", # no z data
##"eso507-g025",
# "ic5193", # no data
# "n0080",
# "n0380",
# "n0679", # no data
# "n0750",
# "n2208", # no g images.
# "n2256", # no DECam images
# "n2329", # no g images
# "n2418",
# "n2569",
# "n3070",
# "n3091",
##"n3308", # should have data
# "n3311",
# "n4825",
# "n4955",
# "n6223", # no g data
# "n6577",
# "n6688", # no g data
# "n6968",
# "n7265", # no data
# "n7274", # no data
# "n7426", # no data
# "n7618", # no data
# "pgc158229",
# "pgc170207",
# "u03353", # no data
# "u03396", # no data
# "u11990", # no data
# "u12517", # no data

#-------SNAP-31 galaxies (17436)
#DECam_data = "/Users/Joe/data/wfc3-17436/DECam/"
#galaxies = [
# "n2653", # No g-band images
##"ic2437", # didn't work (404) but should have data
# "n4782",
# "n5357",
##"wkk6479", #SHOULD HAVE DATA.
##"pgc165984", #same as wkk6479; should have data
# "ic4842",
# "eso153-g003",
# "n3222",
# "n3771",
# "n3172", # No images.
# "ic1153", # No g-band images
# "n6375",
# "n6442",
# "pgc61702", # No images. aka ESO280-G007
# "eso141-g003",
# "u12179",
# "n3343", # No g-band images
# "ic1143", # No g-band images
# "u10918", # no good data

]

In [18]:
for galaxy in galaxies:

    pgc, objName, ra, dec, logd25 = load_info(galaxy) 
    get_decam(DECam_data, bricks, pgc, galaxy, ra, dec, logd25)

naxis =  2360


--2024-06-19 16:28:05--  https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2997m387/legacysurvey-2997m387-image-g.fits.fz
Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.206.107, 128.55.206.108, 128.55.206.111, ...
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12994560 (12M)
Saving to: 'legacysurvey-2997m387-image-g.fits.fz'

     0K .......... .......... .......... .......... ..........  0%  578K 22s
    50K .......... .......... .......... .......... ..........  0% 1.08M 17s
   100K .......... .......... .......... .......... ..........  1% 1.17M 15s
   150K .......... .......... .......... .......... ..........  1% 1.17M 13s
   200K .......... .......... .......... .......... ..........  1% 1.18M 13s
   250K .......... .......... .......... .......... ..........  2% 1.15M 12s
   300K .......... .......... .......... .......... ..........  2%  920K 12s
   350K 

  5350K .......... .......... .......... .......... .......... 42% 1.38M 4s
  5400K .......... .......... .......... .......... .......... 42% 1.26M 4s
  5450K .......... .......... .......... .......... .......... 43% 5.83M 4s
  5500K .......... .......... .......... .......... .......... 43% 1.08M 4s
  5550K .......... .......... .......... .......... .......... 44% 1.64M 4s
  5600K .......... .......... .......... .......... .......... 44% 2.93M 4s
  5650K .......... .......... .......... .......... .......... 44% 1.34M 4s
  5700K .......... .......... .......... .......... .......... 45% 1.65M 4s
  5750K .......... .......... .......... .......... .......... 45% 1.58M 4s
  5800K .......... .......... .......... .......... .......... 46% 1.84M 4s
  5850K .......... .......... .......... .......... .......... 46% 1.32M 4s
  5900K .......... .......... .......... .......... .......... 46% 2.11M 3s
  5950K .......... .......... .......... .......... .......... 47% 2.12M 3s
  6000K ....

 10900K .......... .......... .......... .......... .......... 86% 1.76M 1s
 10950K .......... .......... .......... .......... .......... 86% 1.90M 1s
 11000K .......... .......... .......... .......... .......... 87% 1.13M 1s
 11050K .......... .......... .......... .......... .......... 87% 1.34M 1s
 11100K .......... .......... .......... .......... .......... 87% 1.37M 1s
 11150K .......... .......... .......... .......... .......... 88% 2.45M 1s
 11200K .......... .......... .......... .......... .......... 88%  400K 1s
 11250K .......... .......... .......... .......... .......... 89% 1.18M 1s
 11300K .......... .......... .......... .......... .......... 89% 1.31M 1s
 11350K .......... .......... .......... .......... .......... 89% 2.83M 1s
 11400K .......... .......... .......... .......... .......... 90% 1.26M 1s
 11450K .......... .......... .......... .......... .......... 90% 1.44M 1s
 11500K .......... .......... .......... .......... .......... 91% 1.46M 1s
 11550K ....

  3300K .......... .......... .......... .......... .......... 26%  716K 6s
  3350K .......... .......... .......... .......... .......... 26% 99.9M 6s
  3400K .......... .......... .......... .......... .......... 27% 1.96M 6s
  3450K .......... .......... .......... .......... .......... 27% 1.31M 6s
  3500K .......... .......... .......... .......... .......... 28% 2.17M 6s
  3550K .......... .......... .......... .......... .......... 28% 1.78M 6s
  3600K .......... .......... .......... .......... .......... 28% 1.36M 6s
  3650K .......... .......... .......... .......... .......... 29% 1.30M 5s
  3700K .......... .......... .......... .......... .......... 29% 2.46M 5s
  3750K .......... .......... .......... .......... .......... 30% 1.71M 5s
  3800K .......... .......... .......... .......... .......... 30% 1.39M 5s
  3850K .......... .......... .......... .......... .......... 30% 1.51M 5s
  3900K .......... .......... .......... .......... .......... 31% 1.56M 5s
  3950K ....

...

  3700K .......... .......... .......... .......... .......... 30%  795K 9s
  3750K .......... .......... .......... .......... .......... 31%  779K 9s
  3800K .......... .......... .......... .......... .......... 31%  174K 10s
  3850K .......... .......... .......... .......... .......... 32%  846K 10s
  3900K .......... .......... .......... .......... .......... 32%  839K 10s
  3950K .......... .......... .......... .......... .......... 32%  799K 10s
  4000K .......... .......... .......... .......... .......... 33%  585K 10s
  4050K .......... .......... .......... .......... .......... 33%  855K 10s
  4100K .......... .......... .......... .......... .......... 34%  619K 10s
  4150K .......... .......... .......... .......... .......... 34%  648K 9s
  4200K .......... .......... .......... .......... .......... 34%  650K 9s
  4250K .......... .......... .......... .......... .......... 35%  591K 9s
  4300K .......... .......... .......... .......... .......... 35%  842K 9s
  435

  9150K .......... .......... .......... .......... .......... 75% 1.02M 4s
  9200K .......... .......... .......... .......... .......... 76%  573K 4s
  9250K .......... .......... .......... .......... .......... 76%  589K 4s
  9300K .......... .......... .......... .......... .......... 76% 1.08M 4s
  9350K .......... .......... .......... .......... .......... 77%  448K 4s
  9400K .......... .......... .......... .......... .......... 77%  982K 4s
  9450K .......... .......... .......... .......... .......... 78%  561K 4s
  9500K .......... .......... .......... .......... .......... 78% 1.12M 4s
  9550K .......... .......... .......... .......... .......... 78%  459K 4s
  9600K .......... .......... .......... .......... .......... 79%  626K 4s
  9650K .......... .......... .......... .......... .......... 79%  898K 3s
  9700K .......... .......... .......... .......... .......... 80% 1.05M 3s
  9750K .......... .......... .......... .......... .......... 80% 1.04M 3s
  9800K ....


mv tmp.fits ic4926_g.fits


--2024-06-19 16:29:12--  https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/299/2997m387/legacysurvey-2997m387-image-z.fits.fz
Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.206.111, 128.55.206.109, 128.55.206.107, ...
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.111|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-06-19 16:29:12 ERROR 404: Not Found.

--2024-06-19 16:29:12--  https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/300/3000m387/legacysurvey-3000m387-image-z.fits.fz
Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.206.111, 128.55.206.109, 128.55.206.107, ...
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.111|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-06-19 16:29:13 ERROR 404: Not Found.

--2024-06-19 16:29:13--  https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr10/south/coadd/300/3003m387/legacysurvey-3003m387-image-z.fits.f

FileNotFoundError: [Errno 2] No such file or directory: 'legacysurvey-2997m387-image-z.fits.fz'

In [ ]:
print(logd25)